In [1]:
import torch
import sys

sys.path.append('../')
from src.models.components.xlsr_conformertcm_baseline import Model as XLSRConformerTCM

/home/hungdx/miniconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


ckpt = torch.load(
    '/nvme1/hungdx/Lightning-hydra/logs/train/runs/2025-03-31_09-25-21/checkpoints/epoch_026.ckpt', weights_only=False)


In [4]:
args = ckpt['hyper_parameters']['args']['conformer']

In [5]:


model = XLSRConformerTCM(args=args,
                         ssl_pretrained_path='/nvme1/hungdx/Towards-Real-Time-Deepfake-Speech-Detection-in-Resource-Limited-Scenarios/pretrained/xlsr2_300m.pt')

model

/home/hungdx/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Model(
  (front_end): SSLModel(
    (model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
            (0): Conv1d(512, 512, kernel_size=

In [6]:
from collections import OrderedDict


def load_model_weights(model, checkpoint, required_prefix='net'):
    """
    Load model weights from checkpoint with specific key prefix requirements.
    
    Args:
        model: PyTorch model to load weights into
        checkpoint_path: Path to the checkpoint file
        required_prefix: Required prefix for state dict keys (default: 'net')
        
    Returns:
        model: Model with loaded weights
    """
  
    state_dict = checkpoint if isinstance(
        checkpoint, dict) else checkpoint.state_dict()
    
    print(f"Loading {len(state_dict)} parameters from checkpoint")

    # Create new state dict with processed keys
    new_state_dict = OrderedDict()

    # Process the state dict keys
    for key, value in state_dict.items():
        # Skip keys that don't start with the required prefix
        if not key.startswith(required_prefix):
            continue

        # Remove the 'net.' prefix to match model's state dict keys
        new_key = key[len(required_prefix) + 1:]  # +1 for the dot after prefix
        new_state_dict[new_key] = value

    # Load the processed state dict
    try:
        model.load_state_dict(new_state_dict, strict=True)
        print(f"Successfully loaded {len(new_state_dict)} parameters")
    except RuntimeError as e:
        print(f"Error loading state dict: {str(e)}")
        raise

    return model

model = load_model_weights(model, ckpt['state_dict'])



Loading 589 parameters from checkpoint
Successfully loaded 588 parameters


In [ ]:
# Save the model
torch.save(model.state_dict(), 'MDT_202504.pth')

: 

In [21]:
new_ckpt = torch.load(
    '/datad/pretrained/AudioDeepfakeCMs/S_241214_conf-1.pth')
new_ckpt.keys()

/tmp/ipykernel_2523835/134978345.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_ckpt = torch.load(


odict_keys(['front_end.model.mask_emb', 'front_end.model.feature_extractor.conv_layers.0.0.weight', 'front_end.model.feature_extractor.conv_layers.0.0.bias', 'front_end.model.feature_extractor.conv_layers.0.2.1.weight', 'front_end.model.feature_extractor.conv_layers.0.2.1.bias', 'front_end.model.feature_extractor.conv_layers.1.0.weight', 'front_end.model.feature_extractor.conv_layers.1.0.bias', 'front_end.model.feature_extractor.conv_layers.1.2.1.weight', 'front_end.model.feature_extractor.conv_layers.1.2.1.bias', 'front_end.model.feature_extractor.conv_layers.2.0.weight', 'front_end.model.feature_extractor.conv_layers.2.0.bias', 'front_end.model.feature_extractor.conv_layers.2.2.1.weight', 'front_end.model.feature_extractor.conv_layers.2.2.1.bias', 'front_end.model.feature_extractor.conv_layers.3.0.weight', 'front_end.model.feature_extractor.conv_layers.3.0.bias', 'front_end.model.feature_extractor.conv_layers.3.2.1.weight', 'front_end.model.feature_extractor.conv_layers.3.2.1.bias', 

In [19]:
model.load_state_dict(new_ckpt)

/tmp/ipykernel_2523835/1426476301.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(


<All keys matched successfully>

In [17]:
model

Model(
  (front_end): SSLModel(
    (model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
            (0): Conv1d(512, 512, kernel_size=